In [1]:
from connections.mysql_connector import MySQL_Connector
from models.topic_modeling import Topic_Modeling
from connections.neo4j_connector import Neo4j_Connector
import os
from datetime import datetime
from gensim import corpora, models, similarities
from models.graph_generator import Graph_Generator
from models.tuple_extractor import Tuple_Extractor
from acessos import read, get_conn, persistir_uma_linha, persistir_multiplas_linhas, replace_df
import pandas as pd
import credentials

## Fazendo as conexões

In [3]:
connector = MySQL_Connector("conn_orfeu")
neo4j_client = Neo4j_Connector(credentials.neo4j_uri, credentials.neo4j_user, credentials.neo4j_password)
graph_generator = Graph_Generator(neo4j_client)

tuple_extractor = Tuple_Extractor()

In [9]:
def gerar_tuplas(texto):
    tuple_extractor.extrair_tupla(texto)
    df_tuplas = tuple_extractor.get_ultimas_tuplas_geradas()
    
    return df_tuplas

def gerar_df_doc(texto):
    data =  {'nome': [texto], 'type': ["DOC"]}
    df_doc = pd.DataFrame(data=data)
    return df_doc

def pre_processar_tuplas(df_tuplas):
    df_tuplas['arg1'] = df_tuplas.apply(lambda x: graph_generator.pre_process_text(x['arg1']), axis=1) 
    df_tuplas['arg2'] = df_tuplas.apply(lambda x: graph_generator.pre_process_text(x['arg2']), axis=1) 
    df_tuplas['rel'] = df_tuplas.apply(lambda x: graph_generator.pre_process_text(x['rel'], rel=True), axis=1) 
    
    return df_tuplas

def get_df_sentencas(df_tuplas):
    df_sentencas = pd.DataFrame()
    df_sentencas['nome'] = df_tuplas['sentenca']#.to_frame()
    df_sentencas['type'] = "SENTENÇA"
    
    return df_sentencas

def processar_df_tuplas(df_tuplas):
    df_tuplas['type'] = "ARG1"
    df_arg1 = df_tuplas[['arg1', 'type']]
    df_arg1 = df_arg1.rename(columns={"arg1": 'nome'})
    
    df_tuplas['type'] = "ARG2"
    df_arg2 = df_tuplas[['arg2', 'type']]
    df_arg2 = df_arg2.rename(columns={"arg2": 'nome'})
    
    df_tuplas['type'] = "REL"
    df_rel = df_tuplas[['rel', 'type']]
    df_rel = df_rel.rename(columns={"rel": 'nome'})
    return df_arg1, df_arg2, df_rel

def gerar_relacao_doc_sentenca(texto, df_sentencas):
    #ATENÇÃO usar somente após a criação dos nós sentenças e doc
    for sentenca in df_sentencas.iterrows():
        neo4j_client.criar_relacao_de_para(texto, sentenca[1]['nome'], "DOC", sentenca[1]['type'], "extraiu")

def gerar_relacao_arg1_rel_arg2(df_tuplas):
    #ATENÇÃO usar somente após a criação dos nós arg1, arg2 e rel
    for linha in df_tuplas.iterrows():
        #neo4j_client.criar_relacao_de_para(linha[1]["arg1"], linha[1]['rel'], "ARG1", "REL", "extraiu")
        #neo4j_client.criar_relacao_de_para(linha[1]["rel"], linha[1]['arg2'], "REL", "ARG2", "extraiu")
        neo4j_client.criar_relacao_de_para(linha[1]["a"], linha[1]['arg2'], "REL", "ARG2", "extraiu")

def gerar_relacao_sentenca_arg1(df_tuplas): #estou chamando de relação cabeça
    for linha in df_tuplas.iterrows():
        neo4j_client.criar_relacao_de_para(linha[1]["sentenca"], linha[1]['arg1'], "SENTENÇA", "ARG1", "extraiu")

In [7]:
texto = "João mora com maria. João é muito feliz. Maria tem um cachorro chamado Mike. Mike é muito espuleta"
#texto = "O curso de sistemas de informação, permite ao estudante a resolução de problemas de diferentes áreas de conhecimento. Além disso, ele fornece um forte embasamento matemático e de programação."

df_doc = gerar_df_doc(texto)

df_tuplas = gerar_tuplas(texto)
df_tuplas = pre_processar_tuplas(df_tuplas)
print(df_tuplas.head())
df_sentencas = get_df_sentencas(df_tuplas)
print(df_sentencas.head())

 -> Salvando texto no input.txt
  -> Extraindo Tuplas...
Pronto :D
                                sentenca                  arg1   rel  \
0                 João mora com maria .                  João   mora   
1                  João é muito feliz .                  João      é   
2  Maria tem um cachorro chamado Mike .                 Maria    tem   
3  Maria tem um cachorro chamado Mike .   um cachorro chamado      é   
4                 Mike é muito espuleta                  Mike      é   

                        arg2  
0                 com maria   
1               muito feliz   
2  um cachorro chamado Mike   
3                      Mike   
4            muito espuleta   
                                    nome      type
0                 João mora com maria .   SENTENÇA
1                  João é muito feliz .   SENTENÇA
2  Maria tem um cachorro chamado Mike .   SENTENÇA
3  Maria tem um cachorro chamado Mike .   SENTENÇA
4                 Mike é muito espuleta   SENTENÇA


## Salvando Nós no Neo4j

In [11]:
# persistindo documento
neo4j_client.add_nodes(df_doc)

# persistindo sentenças
neo4j_client.add_nodes(df_sentencas)

#persistindo relação doc-sentença
gerar_relacao_doc_sentenca(texto, df_sentencas)

#Persistindo argumentos 
# df_arg1, df_arg2, df_rel = processar_df_tuplas(df_tuplas)
df_arg1, df_arg2 = processar_df_tuplas(df_tuplas)
neo4j_client.add_nodes(df_arg1)
neo4j_client.add_nodes(df_arg2)
#neo4j_client.add_nodes(df_rel)

#persistindo relação cabeça sentenca-arg1
gerar_relacao_sentenca_arg1(df_tuplas)

#persistindo relacao arg1-rel-arg2
gerar_relacao_arg1_rel_arg2(df_tuplas)

Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco


In [13]:
df_backup = df_tuplas.copy()
df_backup.head()

,"""ID SENTENÇA""",sentenca,"""ID EXTRAÇÃO""",arg1,rel,arg2,"""COERÊNCIA""","""MINIMALIDADE""","""MÓDULO SUJEITO""","""MÓDULO RELAÇÃO""","""MÓDULO ARG2"""
0,1.0,João mora com maria .,1.0,João,mora,com maria,NaN,NaN,1.0,1.0,1.0
1,2.0,João é muito feliz .,1.0,João,é,muito feliz,NaN,NaN,1.0,1.0,1.0
2,3.0,Maria tem um cachorro chamado Mike .,1.0,Maria,tem,um cachorro chamado Mike,NaN,NaN,1.0,1.0,1.0
3,NaN,NaN,2.0,um cachorro chamado,é,Mike,NaN,NaN,1.0,3.0,3.0
4,4.0,Mike é muito espuleta,1.0,Mike,é,muito espuleta,NaN,NaN,1.0,1.0,1.0


In [10]:
import math

def processar_sentence_Nan(df_tuple):
    aux_sentenca = ''
    for key, row in df_tuple.iterrows():
        sentenca = row['sentenca']
        
        if isinstance(sentenca, str)== False and math.isnan(sentenca):
            sentenca = aux_sentenca  
            df_tuplas.loc[key, "sentenca"] = sentenca 
           
        else:
            aux_sentenca = sentenca
   
       
        
processar_sentence_Nan(df_tuplas)

df_tuplas

,sentenca,arg1,rel,arg2,type
0,João mora com maria .,João,mora,com maria,REL
1,João é muito feliz .,João,é,muito feliz,REL
2,Maria tem um cachorro chamado Mike .,Maria,tem,um cachorro chamado Mike,REL
3,Maria tem um cachorro chamado Mike .,um cachorro chamado,é,Mike,REL
4,Mike é muito espuleta,Mike,é,muito espuleta,REL


In [27]:
df_tuplas.loc[2, "sentenca"]

'Maria tem um cachorro chamado Mike . '